# lesson3 系列データで分類・予測させる(RNN, LSTM)
## Section1 解説
### 1.1 RNNとは
**再帰型ニューラルネットワーク(Reccurent Neural Network; RNN)**＝系列データを対象としたモデル

CNNでは平面的な位置関係をふまえた学習が行われていたが、RNNでは系列データの持つ(時間的な)前後関係をふまえた学習を目的としている。

まずRNNのモデルの模式図を見る。

<img src="figures/rnn_unfold.png">

[Y. LeCun, Y. Bengio, and G. Hinton, “Deep learning,” Nature, vol. 521, no. 7553, pp. 436–444, 2015.] : (https://www.nature.com/articles/nature14539)

上図＿左はRNNを表すときによく用いられる。

入力$x$に対して変換$U$を適用した後、**繰り返し$W$を適用し**、最後に変換$V$を行ってから出力している。

この繰り返し(再帰)にあたる部分(図中のループ)を展開してわかりやすくしたものが上図＿右である。

注意すべき点は、**RNNの入力データ$x$は系列データ**であること。

つまり、例えば文章を処理する際には、文章を単語(あるいは文字)の系列であると捉え、各文章1文1文をデータとする。

従って、データセット自体はそうした複数の文章の集まりとなる。

<img src="figures/sentiment.png">

文章に対する判別や予測においては、「ある1文がデータセット、各構成単語が入力データ」とはならない。

***

$x$は系列データなので、それ自体$x_1, x_2,..., x_T$という前後関係を持つ要素から構成される。(先の例で$x_t$が単語(あるいは文字)に対応する)

RNNではこの各要素$x_t$に対して$U, W, V$による変換を繰り返し行っていく。

ここで重要なのは、**各$x_t$は独立に処理されるわけではない**ということ。

$x_t$への変換$W$の適用に当たっては、$t$以前の情報を要約したデータ(上図の$s_t$)を考慮している。

これによって、時間的な構造が反映される。

また、使用されている変換$U, W, V$は各時点では同じものになっている。($U_t, W_t, V_t$ではない)

このことからも各時点が独立に処理されないといえる。

***

このような特徴により、RNNはよく時系列データに対して応用される。

実際、株価の予測モデルや自然言語などへの適用が行われている。

特に、自然言語処理への適用では目覚ましい成果を持つ礼もある。(この後扱う)

### 1.2 RNNの実装
RNNの実装を行う前に1つ補足するのは、

上図において、$V$として行っている出力の線形変換(+活性化)は一般にRNNのモデルには含めない、ということ。

つまり、活性化関数を$f$として、
$$
 o_t = f(Ux_t+Ws_{t-1})
$$
を出力するまでがRNNレイヤーの役割。

RNNレイヤーの直後にDenseレイヤーをつければ、上図と同様のモデルを作成することができる。

RNNレイヤーの実装を見ていく。keras.layers.SimpleRNNを用いるだけで簡単に実装することができる。

主な引数
- units : 出力次元(上図$o_t$の次元)
- activation : 活性化関数
- use_bias : バイアスベクトル( $Ux_t+Ws_{t-1}$ に付け加えるベクトル)を使用するか
- {kernel, reccurent, bias}_initializer : 各パラメータの初期化方法(kernelは上図$U$、reccurentは上図$W$)
- {kernel, reccurent, bias, activity}_regularizer
: 各パラメータの正規化( activityは出力=activationのこと)
- {kernel, reccurent, bias}_constraint : 各パラメータに課す制約
- dropout : 入力についてのdropoutの比率
- reccurent_dropout : 再帰についてのdropoutの比率
- return_sequences : False→出力としては系列の最後の出力($o_t$)のみ、True→出力は完全な系列($o_1, o_2, ...,o_T$)を返す
- return_state : True→出力とともに、最後の状態($s_T$)も返す
- go_backwards : True→入力系列を後ろから処理(出力も逆)
- stateful : True→前バッチの各サンプルに対する最後の状態を、次のバッチのサンプルに対する初期状態として引き継ぐ
- unroll : (高速化のためのオプション) True→再帰が展開され高速化されるが、よりメモリに負荷がかかる(短い系列に適する)


```python
keras.layers.SimpleRNN(units, activation='tanh', use_bias=True,
                       kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros',
                       kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None,
                       kernel_constraint=None, recurrent_constraint=None, bias_constraint=None,
                       dropout=0.0, recurrent_dropout=0.0, return_sequences=False, return_state=False,
                       go_backwards=False, stateful=False, unroll=False)
```

### 1.3 BPTTとRNNの注意点
RNNにおけるBackpropagation(誤差逆伝搬)は、実は時間をさかのぼって誤差を逆伝搬させることに対応している。

つまり、RNN層の勾配(更新のために使用する誤差情報)は

**(RNN層の勾配) = (t=Tの勾配)×・・・×(t=2の勾配)×(t=1の勾配)**

のように、RNN層内の各時点での勾配の積になる。

そこでRNNでのBackpropagationを特に**Backpropagation Through Time(BPTT)とよぶ。

このように、RNNでは時間の分だけ勾配の積が発生するので、出力付近の勾配が過大に、入力付近の勾配が過少になる傾向がある。

これらを**勾配爆発**、**勾配消失**とよぶ。

勾配爆発が起きるとオーバーフローなどで計算が不安定になる。

勾配消失が起きると入り口付近の更新が進まなくなる。

SimpleRNNにはこういった課題も存在(対策は後述)
